## Математическая модель

### Обозначения
- $G = (N, A)$ — транспортная сеть, где $N$ — множество остановок, $A$ — множество рёбер (маршрутов)
- $d_{ij}$ — время в пути по ребру $(i,j) \in A$
- $h_{ij}$ — интервал движения (headway) для маршрута из $i$ в $j$
- $f_{ij} = 1/h_{ij}$ — частота движения (при $h_{ij} > 0$)
- $D$ — пункт назначения
- $u_i$ — ожидаемое время в пути от остановки $i$ до $D$
- $f_i$ — суммарная частота оптимальных маршрутов из остановки $i$

---

## Оригинальный алгоритм Spiess-Florian (1989)

**Цель:** Минимизация ожидаемого времени в пути до пункта назначения $D$.

### Шаг 1: Поиск оптимальной стратегии
Инициализация:
$$
u_i = \begin{cases} 
0 & \text{если } i = D \\
+\infty & \text{иначе}
\end{cases}, \quad f_i = 0 \quad \forall i \in N
$$

Для каждого ребра $(i,j) \in A$ вычисляем приоритет:
$$
\pi_{ij} = u_j + d_{ij}
$$

Алгоритм обновляет метки узлов:
$$
u_i \leftarrow \frac{f_i \cdot u_i + f_{ij} \cdot (u_j + d_{ij})}{f_i + f_{ij}}
$$
$$
f_i \leftarrow f_i + f_{ij}
$$

где $f_{ij} = 1/h_{ij}$ — частота маршрута $(i,j)$.

### Шаг 2: Распределение пассажиропотоков
Объём пассажиров на ребре $(i,j)$:
$$
v_{ij} = \left(\frac{f_{ij}}{f_i}\right) \cdot V_i
$$
где $V_i$ — объём пассажиров в узле $i$.

Уравнение сохранения потока:
$$
V_j = V_j + v_{ij}, \quad V_D = -\sum_{i \neq D} V_i
$$

---

## Математическая модель модифицированного алгоритма Spiess-Florian

### Обозначения
- $G = (N, A)$ — транспортная сеть, где $N$ — множество остановок, $A$ — множество рёбер
- $d_{ij}$ — среднее время в пути по ребру $(i,j)$
- $h_{ij}$ — интервал движения, $f_{ij} = 1/h_{ij}$ — частота (при $h_{ij} > 0$)
- $T$ — максимальное допустимое время в пути (deadline)
- $\mu_i, \sigma_i^2$ — параметры нормального распределения оставшегося времени от $i$ до $D$
- $R_i = P(X_i \leq T) = \Phi\left(\frac{T - \mu_i}{\sqrt{\sigma_i^2}}\right)$ — вероятность прибытия вовремя
- $\Phi$ — функция стандартного нормального распределения

---

## Алгоритм: Максимизация вероятности прибытия вовремя

### Шаг 1: Инициализация
$$
(\mu_i, \sigma_i^2) = 
\begin{cases} 
(0, 0) & \text{если } i = D \\
(+\infty, 0) & \text{иначе}
\end{cases},
\quad
f_i = 0 \quad \forall i \in N
$$

### Шаг 2: Приоритетная очередь с двухуровневым критерием
Для ребра $(i,j)$ вычисляем временные характеристики:
$$
\mu_{\text{tent}} = \underbrace{\frac{1}{f_{ij}}}_{\text{ожидание}} + d_{ij} + \mu_j
$$
$$
\sigma_{\text{tent}}^2 = \underbrace{\frac{1}{f_{ij}}}_{\text{дисперсия ожидания}} + \sigma_j^2
$$
$$
R_{\text{tent}} = \Phi\left(\frac{T - \mu_{\text{tent}}}{\sqrt{\max(\sigma_{\text{tent}}^2, 10^{-8})}}\right)
$$

**Приоритет в очереди:** $(-R_{\text{tent}},\ \mu_{\text{tent}})$  
→ Сначала максимизируем вероятность $R$, при равных $R$ выбираем маршрут с **меньшим ожидаемым временем** $\mu$.

### Шаг 3: Обновление меток узла
При рассмотрении ребра $(i,j)$:

**Если $f_i = 0$ (первый маршрут):**
$$
\mu_i^{\text{new}} = \mu_{\text{tent}}, \quad \sigma_i^{2,\text{new}} = \sigma_{\text{tent}}^2
$$

**Если $f_i > 0$ (агрегация маршрутов):**
$$
\mu_i^{\text{new}} = \frac{f_i \cdot \mu_i + f_{ij} \cdot \mu_{\text{tent}}}{f_i + f_{ij}}
$$
$$
m2_{\text{old}} = \sigma_i^2 + \mu_i^2, \quad m2_{\text{new}} = \sigma_{\text{tent}}^2 + \mu_{\text{tent}}^2
$$
$$
m2_i^{\text{new}} = \frac{f_i \cdot m2_{\text{old}} + f_{ij} \cdot m2_{\text{new}}}{f_i + f_{ij}}
$$
$$
\sigma_i^{2,\text{new}} = \max(m2_i^{\text{new}} - (\mu_i^{\text{new}})^2,\ 0)
$$
$$
f_i^{\text{new}} = f_i + f_{ij}
$$

**Условие обновления:**
$$
R_i^{\text{new}} > R_i^{\text{old}} + 10^{-6} \quad \text{ИЛИ} \quad (R_i^{\text{new}} \approx R_i^{\text{old}} \text{ и } \mu_i^{\text{new}} < \mu_i^{\text{old}})
$$

### Шаг 4: Распределение пассажиропотоков
Объём на ребре $(i,j)$:
$$
v_{ij} = \left(\frac{f_{ij}}{f_i}\right) \cdot V_i
$$
где $V_i$ — объём пассажиров в узле $i$.

